In [9]:
import json
import sys

CorpusFile=open('./data/hi_train_dev_tagged.txt','r')
Corpus=[i.strip().split(' ') for i in CorpusFile.readlines()]
CorpusFile.close()

A={}
B={}
C={}

A['start']={}
A['end']={}

for line in Corpus:
    start=line[0].rsplit('/',1)
    end=line[-1].rsplit('/',1)
    A['start'][start[1]]=A['start'].get(start[1],0)+1
    A['end'][end[1]]=A['end'].get(end[1],0)+1
    B[end[1]]=B.get(end[1],{})
    B[end[1]][end[0]]=B[end[1]].get(end[0],0)+1
    next_word=start
    for i in range(len(line)-1):
        cur_word=next_word
        next_word=line[i+1].rsplit('/',1)
        A[cur_word[1]]=A.get(cur_word[1],{})
        A[cur_word[1]][next_word[1]]=A[cur_word[1]].get(next_word[1],0)+1
        B[cur_word[1]]=B.get(cur_word[1],{})
        B[cur_word[1]][cur_word[0]]=B[cur_word[1]].get(cur_word[0],0)+1  
totalTrans=len(B)
totalWords=0

for k in B:
    totalWords=totalWords+len(B.get(k))
    for word in B.get(k):
        C[word]=C.get(word,0)
        
for k in A:
    items=A.get(k)
    lenOfItems=len(items)
    total=sum(items.values())
    pTrans=1/(total+lenOfItems)
    for trans in B.keys():
        if trans in items:
            A[k][trans]=A[k][trans]/total-pTrans
        else:
            A[k][trans]=lenOfItems*pTrans/(totalTrans-lenOfItems)       
for k in B:
    if not k in A:
        A[k]={x:1/len(B.keys()) for x in B.keys()}
    items=B.get(k)
    total=sum(items.values())
    pEm=1/(total+len(items))
    for em in items:
        B[k][em]=B[k][em]/total-pEm
    B[k]['other']=len(items)*pEm/(totalWords-len(items))
    B[k]['items']=len(B[k])
    B[k]['total']=sum(B[k].values())
        
Model=[A,B,C]
ModelFile=open('./hmmmodel.txt','w')
ModelFile.write(json.dumps(Model))
ModelFile.close()

In [10]:
import json
import sys

InputFile=open('./data/hi_test_raw.txt','r')
Input=[i.strip().split(' ') for i in InputFile.readlines()]


ModelFile=open('./hmmmodel.txt')
Model=json.loads(ModelFile.read())
ModelFile.close()

A=Model[0]
B=Model[1]
C=Model[2]
BK=B.keys()

output=''
for sentence in Input:
    viterbi={}
    backpointer={}
    viterbi[0]={}
    backpointer[0]={}
    #Initialization
    em=0
    es=0
    if not sentence[0] in C and len(sentence[0])<=2:
        for word in B['NN']:
            if sentence[0][-1]==word[-1]:
                em=em+B['NN'][word]
    if not sentence[0] in C and sentence[0][-1]=='s' and 'NNS' in B:
        for word in B['NNS']:
            if word[-1]=='s':
                es=es+B['NNS'][word]
    for state in BK:
        emB=1/len(BK)
        if not sentence[0] in B[state]:
            emB=B[state]['other']
            if state=='NN':
                emB=em
            if state=='NNS':
                emB=es
            if sentence[0][-1].isdigit() or sentence[0][0].isdigit() or sentence[0][-1]=='%':
                if(state=='CD'):
                    emB=1
                else:
                    emB=0
            if sentence[0][0:4]==('http' or 'www') or sentence[0][-3:]=='com':
                if(state=='ADD'):
                    emB=1
                else:
                    emB=0
        else:
            emB=B[state][sentence[0]]
        viterbi[0][state]=A['start'][state]*emB
        backpointer[0][state]='start'
    for t in range(len(sentence)-1):
        viterbi[t+1]={}
        backpointer[t+1]={}
        em=0
        es=0
        if not sentence[t+1] in C and len(sentence[t+1])<=2:
            for word in B['NN']:
                if sentence[t+1][-1]==word[-1]:
                    em=em+B['NN'][word]
        if not sentence[t+1] in C and sentence[t+1][-1]=='s' and 'NNS' in B:
            for word in B['NNS']:
                if word[-1]=='s':
                    es=es+B['NNS'][word]
        for state in BK:
            maxK=''
            maxV=0
            emB=0
            if not sentence[t+1] in B[state]:
                emB=B[state]['other']
                if state=='NN':
                    emB=em
                if state=='NNS':
                    emB=es
                if sentence[t+1][-1].isdigit() or sentence[t+1][0].isdigit() or sentence[t+1][-1]=='%':
                    if(state=='CD'):
                        emB=1
                    else:
                        emB=0
                if sentence[t+1][0:4]==('http' or 'www') or sentence[t+1][-3:]=='com':
                    if(state=='ADD'):
                        emB=1
                    else:
                        emB=0
            else:
                emB=B[state][sentence[t+1]]
            for s in BK:
                V=viterbi[t][s]*A[s][state]*emB
                if V>=maxV:
                    maxV=V
                    maxK=s
            viterbi[t+1][state]=maxV
            backpointer[t+1][state]=maxK
    maxK=''
    maxV=0
    for s in BK:
        V=viterbi[len(sentence)-1][s]*A['end'][s]
        if V>=maxV:
            maxV=V
            maxK=s
    viterbi['end']=maxV
    backpointer['end']=maxK
    tag=backpointer['end']
    for i in range(len(sentence)):
        sentence[-i-1]=sentence[-i-1]+'/'+tag+' '
        tag=backpointer[len(sentence)-i-1][tag]
    output=output+''.join(sentence).strip()+'\n'
OutputFile=open('./hmmoutput.txt','w', encoding='utf8')
OutputFile.write(output)
OutputFile.close()

In [11]:
OutputFile=open('./hmmoutput.txt','r')
StandardFile=open('./data/hi_test_tagged.txt','r')
X=[i.strip().split(' ') for i in OutputFile.readlines()]
Y=[i.strip().split(' ') for i in StandardFile.readlines()]
OutputFile.close()
StandardFile.close()
Stat={}
total=0
incorrect=0
for i in range(len(X)):
    for j in range(len(X[i])):
        total=total+1
        if not X[i][j]==Y[i][j]:
            incorrect=incorrect+1
            x=X[i][j].rsplit('/',1)[1]
            y=Y[i][j].rsplit('/',1)[1]
            #if(x=='NN' and y=='NNP'):
             #   print('F: '+X[i][j]+' T: '+Y[i][j]+'\n')
            Stat[y]=Stat.get(y,{})
            Stat[y][x]=Stat[y].get(x,0)+1
print(1-incorrect/total)


0.8834885690093142


In [554]:
{k:sum(v.values()) for (k,v) in Stat.items()}

{"''": 14,
 ',': 33,
 '.': 9,
 ':': 11,
 'ADD': 11,
 'AFX': 4,
 'CC': 16,
 'CD': 10,
 'DT': 58,
 'EX': 6,
 'FW': 22,
 'GW': 30,
 'HYPH': 16,
 'IN': 112,
 'JJ': 272,
 'JJR': 6,
 'JJS': 16,
 'MD': 6,
 'NFP': 22,
 'NN': 737,
 'NNP': 339,
 'NNPS': 39,
 'NNS': 133,
 'POS': 5,
 'PRP': 14,
 'PRP$': 3,
 'RB': 233,
 'RBR': 14,
 'RBS': 7,
 'RP': 24,
 'SYM': 10,
 'TO': 16,
 'UH': 48,
 'VB': 158,
 'VBD': 87,
 'VBG': 92,
 'VBN': 100,
 'VBP': 105,
 'VBZ': 38,
 'WDT': 25,
 'WP': 2,
 'WP$': 2,
 'WRB': 5,
 'XX': 3,
 '``': 14}

In [556]:
Stat['NNS']

{'.': 1,
 'CD': 9,
 'JJ': 12,
 'NN': 7,
 'NNP': 68,
 'NNPS': 3,
 'PRP': 1,
 'RB': 10,
 'VB': 3,
 'VBD': 2,
 'VBN': 11,
 'VBP': 1,
 'VBZ': 4,
 'XX': 1}

In [324]:
C['0']

{'(': 0.008507625429054365,
 'BB': 0.0017525591293295712,
 'CD': 0.1361033222016338,
 'DT': 0.0582949659344334,
 'FW': 0.005755561381018338,
 'IN': 0.12909806826117845,
 'JJ': 0.017264192854623543,
 'LS': 0.0005016209256768318,
 'MD': 0.0012521838478684755,
 'NN': 0.2119101773429898,
 'NNB': 1.2456442157358569e-06,
 'NNP': 0.22742181106828377,
 'PFA': 0.008507625429054365,
 'PFN': 1.2456442157358569e-06,
 'PRD': 0.007006499584671077,
 'PRP': 0.05479233896420573,
 'RB': 0.06029646706027778,
 'VC': 1.2456442157358569e-06,
 'VV': 0.06379909403050545,
 '``': 0.002753309692251763,
 'other': 0.0002263109059225564}